# Week 10

## Objectives so far

* Start working on a RL backbone for the bead model
* Code a graph generator for Unity
* integrate the force fields with the Unity polymer chain 

## August 16

Today I have two large objectives and a tentative one:
* **1.** Write the description of the MARTINI model in the Google Doc and send it to Paloma.
* **2.** Check if it is possible to keep running the simulation with the forces attached. In addition to this, check what is the procedure with the dihedral force.
* **3.** Start coding the ML-Agents reinforcement learning architecture.

Unfortunately, for most of the day the work has been focused mostly on making the Unity simulation work in my laptop, which for some reason is not yielding results.

## August 17

So yesterday (August 16th), I found out with Paloma that the MARTINI simulation is apparently not working on my laptop. It could be a Windows thing, but I set myself to find out why this is happening and what can be done to make it work. In addition to this, I decided to start looking into how to integrate this myself. Yesterday I tried to do this from Paloma's computer and it did not quite work; especially, because the `BondedAngle.cs` and `BondedDihedral.cs` files did not initialize along with the generation of the MARTINI chain. I need to fix that.

The following text is what I have for the paper so far:

[Check the Google Doc](https://docs.google.com/document/d/1AwGT2lUVPjqtmVabAxmrEw-v3aw5f0tjkJtZvyaWPGg/edit)

## August 18

### Meeting (8/18/2022 - 10:00am)

We decided the further course of action from the project from now. There are a couple things from now to code:

* **1.** We need to get the code working in all computers, as it for now, it only seems to be working on Mac.
* **2.** We need to prepare a reinforcement learning simulation. It does not have to be perfect as it is, but it has to at least yield some sort of result.
* **3.** We need to write the paper.

This day, we managed to get the code working in all computers by replacing the line `itp["atoms"]["smiles"]` with the key index of the `"smiles"` string in the dictionary. This might be the result of the parser not interpreting the lines as strings in some OS or versions of Unity. Nonetheless, we are glad that we made it work.

Through multiple efforts, we also managed to get the `BondedDihedral.cs` file working. Finally, we managed to start a small RL simulation. In order to make the simulation work we decided on the following architecture:

* **1.** We are going to choose to focus only in the start and ending beads of each monomer. Through Agent observations we will obtain the dihedral angle between them and the bond angle.
* **2.** For actions, we will set a random force on the beads.
* **3.** We will reward on the difference between the estimated force and the calculated force on each steps.
* **4.** The episode will always end in a predetermined number of steps.

## August 19

Today, we continue the construction of the reinforcement learning architecture. The template for the paper has already been set up by Paloma and is available on Overleaf [here](https://www.overleaf.com/project/62fe95a9f741fdfdd1beda14).

Andy has been working on setting up the RL environment through ML-Agents in order to set up a training phase. For this reason, I will be working on setting up the length-scale and the time-scale inside Unity in order for it to be accurate.

## August 20

Repurposing of `BondedPotential.cs` for a higher level structure

```c#
using System.Collections;
using System.Collections.Generic;
using UnityEngine;
using System;

public class BondedPotential : MonoBehaviour
{
    /// This collection contains all lists of dihedrals in the MARTINI chain.
    public List< List<GameObject> > bonds = new();

    /// This dictionary will contain the molecule in a more accesible graph format
    public static Dictionary< GameObject, List<GameObject> > graph = new();

    /// This dictionary will provide the values for K_b and d_b
    public static Dictionary< int, List<float> > values = new();

    /// <summary>
    /// Find all the non-repeating list of atoms that form a bond angle.
    /// Takes arguments in the form <code>(beads, bonds)</code>
    /// </summary>
    /// <param name="graph"> Graph representation of the beads </param>
    /// <param name="bead"> Starting bead from which construct the path </param>
    /// <param name="length"> Length of the sub-path in the graph </param>
    /// <param name="paths"> List of pats </param>
    /// <param name="path"> Current path </param>
    public static void findBonds (Dictionary< GameObject, List<GameObject> > graph, GameObject bead, int length, List< List<GameObject> > paths, List<GameObject> path = null) {
        path = path ?? new List<GameObject>();
        if (path != null) {
            path.Add(bead);
            List<GameObject> pathr = new List<GameObject>(path);
            pathr.Reverse();
            if (path.Count == length) {
                if (!paths.Contains(path) && !paths.Contains(pathr)) {
                    paths.Add(path);
                }
            }
            else {
                foreach (GameObject neighbour in graph[bead]) {
                    List<GameObject> copy = new List<GameObject>(path);
                    findBonds(graph, neighbour, length, paths, copy);
                }
            }
        }
    }

    // List of all the neighbours to the current bead
    static public List<GameObject> bondedNeighbours;

    /// <summary>
    /// Calculates the Bonded force between 2 beads
    /// Takes arguments in the form <code>(r_ij, K_b, d_b)</code>
    /// </summary>
    /// <param name="Kb"> Vector3 from bead i to bead j </param>
    /// <param name="db"> The force constant </param>
    /// <param name="distances"> The equilibrium distance between two beads </param>
    /// <summary>
    /// Computes the bond angle force
    /// Takes arguments in the form <code>(angle_cos, angle_beads, distances)</code>
    /// It returns a tuple <code>(F1, F2, F3, F4)</code> with the vector forces corresponding to each bead
    /// </summary>
    /// <param name="dangle_cos"> The cosine of the dihedral input </param>
    /// <param name="dangle_beads"> The list of beads that form the dihedral </param>
    /// <param name="distances"> The list of Vector3 containing the positions of the beads that form the dihedral </param>
    public static List<Vector3> F_b(float Kb, List<GameObject> db, List<Vector3> distances) {
        Vector3 r = distances[1] - distances[0];

        Vector3 F1 = -2f * Kb * (r.magnitude - db) * r/r.magnitude;
        Vector3 F2 = -2f * Kb * (r.magnitude - db) * r/r.magnitude;

        List<Vector3> result = new List<Vector3>();
        result.Add(F1); result.Add(F2); 
        return result;
    }

    void Start()
    {
        MartiniModel chain = gameObject.GetComponent<MartiniModel>();
        values = chain.PBONDS;

        // We construct a graph representation of the entire molecule by
        // making a dictionary with structure
        // keyBead : [neighbourBeads]
        for (int i = 0; i < chain.BEADS.Count; i++) {
            graph[chain.BEADS.ElementAt(i)] = chain._edges[i].Select(x => chain._beads[x]).ToList();
        }

        // We use the findAngles function to get the subpaths of size n that we want
        List< List<GameObject> > paths = new List< List<GameObject> >();
        int length = 2;
        foreach (GameObject bead in graph.Keys) {
            findBonds(graph, bead, length, paths);
        }
        bonds = paths;
    }

    void FixedUpdate()
    {
        for (int i = 0; i < dihedrals.Count; i++) {
            List<GameObject> bond_beads = bonds[i];

            bond_beads[0].GetComponent<RigidBody>().AddForce(F[0]);
            bond_beads[1].GetComponent<RigidBody>().AddForce(F[0]);

            Vector3 one = (bond_beads[0].transform.position);
            Vector3 two = (bond_beads[1].transform.position);

            // Computes the values of K_b and d_b
            float K_b = values[current.GetComponent<EmbeddedBead>.Index, neighbour.GetComponent<EmbeddedBead>.Index][1] * 1000;
            float d_b = values[current.GetComponent<EmbeddedBead>.Index, neighbour.GetComponent<EmbeddedBead>.Index][0];
            List<Vector3> vectors = new List<Vector3>();
            vectors.Add(one); vectors.Add(two);

            // Computer the force of the potential angle
            List<Vector3> F = F_b(K_b, d_b, vectors);

            Debug.Log(string.Format("{0}: {1}", i, string.Join(" , ", F)));

            // Applies the force calculated to each bead in the dihedral
            bond_beads[0].GetComponent<Rigidbody>().AddForce(F[0]);
            bond_beads[1].GetComponent<Rigidbody>().AddForce(F[1]);
        }
    }
}
``` 

---

Repurposing `MartiniModel.cs`

```c#
using System;
using System.Collections.Generic;
using System.Dynamic;
using System.Linq;
using MD.Parser;
using Python.Runtime;
using UnityEngine;

public class MartiniModel : ChainCreator
{
    public static MartiniModel Instance;

    // VdW diameter for:
    // normal: 5.2 angstroms
    // small (in a ring, prefixed by 'S'): 4.7

    // Dictionary of beads ordered by the index given by the .itp file
    public Dictionary<int, GameObject> _beads = new();
    // Dictionary of pairs of indexes to bond GameObjects
    public Dictionary<(int, int), GameObject> _bonds = new();
    public Dictionary<int, List<int>> _edges = new();
    public Dictionary< List<int>, List<float> > _dihedrals = new();
    public Dictionary< List<int>, List<float> > _angles = new();
    public Dictionary< int, List<float> > _pbonds = new();

    public List<(GameObject, GameObject)> _pairs = new();
    public GameObject AgentModel = null;

    public override List<GameObject> BEADS
    {
        get => new List<GameObject>(_beads.Values);
    }

    public override Dictionary<(int, int), GameObject> BONDS
    {
        get => _bonds;
    }

    public override List< (GameObject, GameObject) > PAIRS 
    {
        get => _pairs;
    }

    public override List< List<int>, List<float> > DIHEDRALS {
        get => _dihedrals;
    }

    public override List< List<int>, List<float> > ANGLES {
        get => _angles;
    }

    public override List< int, List<float> > PBONDS {
        get => _pbonds;
    }

    private GameObject beadFolder;
    private GameObject bondFolder;

    public int numMonomer = 10;

    override void Start() {
        Instance = this;
    }

    public override void IngestFormula(GameObject owner, dynamic formula)
    {
        var itp = ITPParser.read((string)formula+".itp");

        List<String> itpKeys = itp.Keys.ToList();
        List<String> atomKeys = new();

        for (int i = 0; i < itp.Keys.Count; i++) {
            if (itpKeys[i] == "atoms") {
                atomKeys = itp["atoms"].Keys.ToList();
            }
        }

        var gro = GROParser.read((string)formula + "_CG.gro");
        owner.name = "CG Chain: " + itp["moleculetype"]["molname"][0];
        
        beadFolder = new GameObject("Beads");
        bondFolder = new GameObject("Bonds");
        bondFolder.transform.parent = owner.transform;
        beadFolder.transform.parent = owner.transform;

        // create beads,
        // TODO: break out into own function (declared below)
        int numBeads = itp["atoms"]["id"].Count;
        // initial monomer
        for (int i = 0; i < numBeads; i++)
        {
            dynamic bead = new ExpandoObject();
            bead.smiles = itp["atoms"][atomKeys[7]][i]; //what the fuck
            bead.s_type = itp["atoms"]["type"][i];
            bead.i = i;
            bead.pos = gro[i];

            // create the bead
            var beadObj = CreateBead(bead, i);
        }

        int numAngles = itp["angles"]["i"].Count;
        for (int i = 0; i < numAngles; i++) {
            List<int> bond_angle = new List<int> {int.Parse(itp["angles"]["i"]), int.Parse(itp["angles"]["j"]), int.Parse(itp["angles"]["k"])};
            _angle[bond_angle] = new List<float> {float.Parse(itp["angles"]["angle"]), float.Parse(itp["angles"]["force.c."])};
        }

        int numDihedrals = itp["dihedrals"]["i"].Count;
        for (int i = 0; i < numAngles; i++) {
            List<int> bond_angle = new List<int> {int.Parse(itp["dihedrals"]["i"]), int.Parse(itp["dihedrals"]["j"]), int.Parse(itp["dihedrals"]["k"]), int.Parse(itp["dihedrals"]["l"])};
            _angle[bond_angle] = new List<float> {float.Parse(itp["dihedrals"]["angle"]), float.Parse(itp["dihedrals"]["force.c."])};
        }

        int numBonds = itp["bonds"]["i"].Count;
        for (int i = 0; i < numBonds; i++) {
            List<int> bondN = new List<int> {int.Parse(itp["bonds"]["i"]), int.Parse(itp["bonds"]["j"])};
            _pbonds[bondN] = new List<float> {float.Parse(itp["bonds"]["length"])*10, float.Parse(itp["bonds"]["force.c."])};
        }
        
        // create bonds
        foreach (var (i, j) in itp["bonds"]["i"].Zip(itp["bonds"]["j"], (a, b) => (int.Parse(a) - 1, int.Parse(b) - 1)))
            CreateBond(i, j);
        
        foreach (var (i, j) in itp["constraints"]["i"].Zip(itp["constraints"]["j"], (a, b) => (int.Parse(a) - 1, int.Parse(b) - 1)))
            CreateBond(i, j);

        var offset = _beads[numBeads - 2].transform.position;
        offset.x -= _beads[0].transform.position.x;
        offset.z -= _beads[0].transform.position.z;

        var monomerDir = (_beads[numBeads - 1].transform.position - _beads[0].transform.position).normalized;

        offset += monomerDir * 2.46f;

        for (int i = 1; i < numMonomer; i++)
        {
            for (int j = 0; j < numBeads; j++)
            {
                dynamic bead = new ExpandoObject();
                bead.smiles = itp["atoms"][atomKeys[7]][j];
                bead.s_type = itp["atoms"]["type"][j];
                bead.i = j + numBeads*i;
                bead.pos = gro[j] + offset*i;

                // create the bead
                var beadObj = CreateBead(bead, j);
            }
            
            CreateBond(_beads.Count - numBeads - 2, _beads.Count - numBeads); // hard coded polymerization
        
            // create bonds
            foreach (var (f, t) in itp["bonds"]["i"].Zip(itp["bonds"]["j"], (a, b) => (int.Parse(a) - 1, int.Parse(b) - 1)))
                CreateBond(_beads.Count - numBeads + f, _beads.Count - numBeads + t);
        
            foreach (var (f, t) in itp["constraints"]["i"].Zip(itp["constraints"]["j"], (a, b) => (int.Parse(a) - 1, int.Parse(b) - 1)))
                CreateBond(_beads.Count - numBeads + f, _beads.Count - numBeads + t);

            if (AgentModel != null)
            {
                // put ML setup here!
                var endIdx = _beads.Count - numBeads - 2;
                var startIdx = _beads.Count - numBeads;
                var aIdx = _edges[startIdx].Except(new[] { endIdx }).First();
                var bIdx = _edges[endIdx].Except(new[] { startIdx }).First();

                _pairs.Add((_beads[startIdx], _beads[endIdx]));

                var startAgent = Instantiate(AgentModel, _beads[startIdx].transform).GetComponent<BeadAgent>();
                var endAgent = Instantiate(AgentModel, _beads[endIdx].transform).GetComponent<BeadAgent>();

                startAgent.creator = gameObject.GetComponent<NewMoleculeCreator>();
                endAgent.creator = gameObject.GetComponent<NewMoleculeCreator>();

                startAgent.idx = startIdx;
                endAgent.idx = endIdx;

                startAgent.dihedral = new() { _beads[aIdx], _beads[startIdx], _beads[endIdx], _beads[bIdx] };
                endAgent.dihedral = new(startAgent.dihedral);
                endAgent.dihedral.Reverse();
                startAgent.bondAngle = new() { _beads[aIdx], _beads[startIdx], _beads[endIdx] };
                endAgent.bondAngle = new() { _beads[startIdx], _beads[endIdx], _beads[bIdx] };
                var rgBodies = new SortedList<int, GameObject>(_beads)
                    .Select(x => x.Value.GetComponent<Rigidbody>()).ToList();
                var embBeads = new SortedList<int, GameObject>(_beads)
                    .Select(x => x.Value.GetComponent<EmbeddedBead>()).ToList();
                startAgent.rgBody = _beads[startIdx].GetComponent<Rigidbody>();
                endAgent.rgBody = _beads[endIdx].GetComponent<Rigidbody>();

                startAgent.exclude = _edges[startIdx].Concat(new []{startIdx}).Select(x => new List<int>(x).Concat(_edges[x])).SelectMany(x => x)
                    .Select(x => _beads[x]).ToList();
                endAgent.exclude = _edges[endIdx].Concat(new []{endIdx}).Select(x => new List<int>(x).Concat(_edges[x])).SelectMany(x => x)
                    .Select(x => _beads[x]).ToList();

                _beads[startIdx].GetComponent<EmbeddedBead>().Cat = MyType.START;
                _beads[endIdx].GetComponent<EmbeddedBead>().Cat = MyType.END;
            }
        }
    }



    public override GameObject CreateBead(dynamic atom, int idx)
    {
        int i = atom.i;
        string smiles = atom.smiles;
        string s_type = atom.s_type;
        Vector3 pos = atom.pos;
        // create the bead
        var beadObj = GameObject.CreatePrimitive(PrimitiveType.Sphere);
        beadObj.name = String.Format("{0}: {1}, {2}",i, s_type, smiles);
        beadObj.transform.parent = beadFolder.transform;
        beadObj.transform.position = pos;
        beadObj.tag = "Bead";
            
        // Attach components
        var embBead = beadObj.GuaranteeGetComponent<EmbeddedBead>();
        //beadObj.AddComponent<NonBondedForce>().chain = this;
            
        // bead information
        {
            embBead.Index = idx;
            var (type, subtype) = (s_type[^2], s_type[^1]);
            embBead.Type = type.ToString();
            embBead.SubType = subtype.ToString();
            if (s_type.Length > 2) embBead.Size = s_type[0].ToString();
        }

        beadObj.transform.localScale = s_type[0] == 'S' ? new Vector3(4.7f,4.7f,4.7f)/2 : new Vector3(5.2f,5.2f,5.2f)/2;
            
        // color the bead with what is basically a random color based on bead type. 
        {
            var hash = new Hash128(); foreach (var chr in s_type) hash.Append(chr);
            ColorUtility.TryParseHtmlString("#" + hash.ToString().Substring(0, 6), out Color color);
            beadObj.GetComponent<Renderer>().material.color = color;
        }

        using (Py.GIL())
        {
            dynamic chem = Py.Import("rdkit.Chem");
            dynamic descriptors = Py.Import("rdkit.Chem.Descriptors");
                
            // get mass
            var rgBody = beadObj.GuaranteeGetComponent<Rigidbody>();
            rgBody.useGravity = false;
            rgBody.mass = (float) descriptors.MolWt(chem.MolFromSmiles(smiles));
            MASS += rgBody.mass;
            rgBody.collisionDetectionMode = CollisionDetectionMode.Continuous;
            //rgBody.detectCollisions = false;
        }

        _beads[i] = beadObj;
        _edges[i] = new List<int>();

        return beadObj;
    }

    public override GameObject CreateBond(dynamic from, dynamic to)
    {
        var i = (int)from;
        var j = (int)to;

        if (_bonds.ContainsKey((i, j))) return _bonds[(i,j)];
        
            
        var fBead = _beads[i]; var tBead = _beads[j];
        
        var bondParentObj = new GameObject(string.Format("{0} -> {1}", i, j));
        bondParentObj.tag = "Bond";
        bondParentObj.transform.position = Vector3.Lerp(fBead.transform.position, tBead.transform.position, 0.5f);
        bondParentObj.transform.parent = bondFolder.transform;

        var bondObj = GameObject.CreatePrimitive(PrimitiveType.Cylinder);
        bondObj.transform.localScale = new Vector3(0.1f, 0.5f, 0.1f);
        bondObj.transform.localRotation = Quaternion.Euler(90f, 0f, 0f);
        bondObj.transform.position = bondParentObj.transform.position;
        bondObj.transform.parent = bondParentObj.transform;
        var rgBody = bondObj.GuaranteeGetComponent<Rigidbody>();
        rgBody.detectCollisions = false;
        rgBody.useGravity = false;

        bondParentObj.transform.LookAt(tBead.transform);
        bondParentObj.transform.localScale = new Vector3(1f, 1f,
            Vector3.Distance(fBead.transform.position, tBead.transform.position));

        var fJoint = bondObj.AddComponent<HingeJoint>();
        fJoint.anchor = -Vector3.up;
        fJoint.connectedBody = fBead.GetComponent<Rigidbody>();
        fJoint.enableCollision = true;

        var tJoint = bondObj.AddComponent<HingeJoint>();
        tJoint.anchor = Vector3.up;
        tJoint.connectedBody = tBead.GetComponent<Rigidbody>();
        tJoint.enableCollision = true;
            
        //Physics.IgnoreCollision(fBead.GetComponent<Collider>(), tBead.GetComponent<Collider>());

        _bonds[(i, j)] = bondParentObj;
        _bonds[(j, i)] = bondParentObj;
        
        _edges[i].Add(j);
        _edges[j].Add(i);

        return bondParentObj;
    }
}
```

---

Repurposing `NonBondedForce.cs`

```c#
using System.Collections;
using System.Linq;
using System.Collections.Generic;
using UnityEngine;
using System;

public class NonBondedForce : MonoBehaviour
{
    public List<GameObject> bondedAtoms;
    public List<float> potentials;
    public List<float> distances;
    public Dictionary<GameObject, EmbeddedBead> embBeads = new();
    public ChainCreator chain;

    // NOTE: Keep constant members of classes top-level and static.
    // This is more memory efficient and will scale up better as we
    // add more of these forces.

    // We compute the Interaction Matrix
    public static IReadOnlyDictionary<(string, string), int> typeMappings = new Dictionary<(string,string), int>() {
            {("Q", "da"), 0}, {("Q", "d"), 1}, {("Q", "a"), 2}, {("Q", "0"), 3},
            {("P", "5"), 4}, {("P", "4"), 5}, {("P", "3"), 6}, {("P", "2"), 7}, {("P", "1"), 8},
            {("N", "da"), 9}, {("N", "d"), 10}, {("N", "a"), 11}, {("N", "0"), 12},
            {("C", "5"), 13}, {("C", "4"), 14}, {("C", "3"), 15}, {("C", "2"), 16}, {("C", "1"), 17},
            {("BP", "4"), 18}
        };

    public static readonly string[,] levels = new string[,]
    {
        {"O", "O", "O", "II", "O", "O", "O", "I", "I", "I", "I", "I", "IV", "V", "VI", "VII", "IX", "IX", "O"},
        {"O", "I", "O", "II", "O", "O", "O", "I", "I", "I", "III", "I", "IV", "V", "VI", "VII", "IX", "IX", "O"},
        {"O", "O", "I", "II", "O", "O", "O", "I", "I", "I", "I", "III", "IV", "V", "VI", "VII", "IX", "IX", "O"},
        {"II", "II", "II", "IV", "I", "O", "I", "II", "III", "III", "III", "III", "IV", "V", "VI", "VII", "IX", "IX", "O"},
        {"O", "O", "O", "I", "O", "O", "O", "O", "O", "I", "I", "I", "IV", "V", "VI", "VI", "VII", "VIII", "O"},
        {"O", "O", "O", "O", "O", "I", "I", "II", "II", "III", "III", "III", "IV", "V", "VI", "VI", "VII", "VIII", "O"},
        {"O", "O", "O", "I", "O", "I", "I", "II", "II", "II", "II", "II", "IV", "IV", "V", "V", "VI", "VII", "O"},
        {"I", "I", "I", "II", "O", "II", "II", "II", "II", "II", "II", "II", "III", "IV", "IV", "V", "VI", "VII", "O"},
        {"I", "I", "I", "III", "O", "II", "II", "II", "II", "II", "II", "II", "III", "IV", "IV", "IV", "V", "VI", "O"},
        {"I", "I", "I", "III", "I", "III", "II", "II", "II", "II", "II", "II", "IV", "IV", "V", "VI", "VI", "VI", "O"},
        {"I", "III", "I", "III", "I", "III", "II", "II", "II", "II", "III", "II", "IV", "IV", "V", "VI", "VI", "VI", "O"},
        {"I", "I", "III", "III", "I", "III", "II", "II", "II", "II", "II", "III", "IV", "IV", "V", "VI", "VI", "VI", "O"},
        {"IV", "IV", "IV", "IV", "IV", "IV", "IV", "III", "III", "IV", "IV", "IV", "IV", "IV", "IV", "IV", "V", "VI", "O"},
        {"V", "V", "V", "V", "V", "V", "IV", "IV", "IV", "IV", "IV", "IV", "IV", "IV", "IV", "IV", "V", "V", "O"},
        {"VI", "VI", "VI", "VI", "VI", "VI", "V", "IV", "IV", "V", "V", "V", "IV", "IV", "IV", "IV", "V", "V", "O"},
        {"VI", "VI", "VI", "VI", "VI", "VI", "V", "IV", "IV", "V", "V", "V", "IV", "IV", "IV", "IV", "V", "V", "O"},
        {"VII", "VII", "VII", "VII", "VI", "VI", "V", "V", "IV", "VI", "VI", "VI", "IV", "IV", "IV", "IV", "IV", "IV", "O"},
        {"IX", "IX", "IX", "IX", "VII", "VII", "VI", "VI", "V", "VI", "VI", "VI", "V", "V", "V", "IV", "IV", "IV", "O"},
        {"IX", "IX", "IX", "IX", "VIII", "VIII", "VII", "VII", "VI", "VI", "VI", "VI", "VI", "V", "V", "IV", "IV", "IV", "O"},
        {"O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"}
    };

    // Interaction Levels with their numeric values assigned to them in a scale [ε] = kJ/mol
    public static IReadOnlyDictionary<string, float> interactionLevels = new Dictionary<string, float>() {
            {"O", 5.6f}, {"I", 5f}, {"II", 4.5f}, {"III", 4f}, {"IV", 3.5f},
            {"V", 3.1f}, {"VI", 2.7f}, {"VII", 2.3f}, {"VIII", 2.0f}, {"IX", 2.0f}
        };

    /// <summary>
    /// Calculates the Lennard-Jones force between 2 entities.
    /// Takes arguments in the form <code>(r, sigma, epsilon)</code>
    /// </summary>
    /// <param name="r">distance between two entities.</param>
    /// <param name="sigma">the sigma vairable</param>
    /// <param name="epsilon">the epsilon variable</param>
    public static Func<float, float, float, float> F_LJ = 
        (r, sigma, epsilon) => 
            48 * epsilon * Mathf.Pow(sigma, 12) / Mathf.Pow(r, 13) - 24 * epsilon * Mathf.Pow(sigma, 6) / Mathf.Pow(r, 7);

    private float interactionLevel(GameObject one, GameObject two) {
        // We get the embedded info
        EmbeddedBead beadOne = embBeads[one];
        EmbeddedBead beadTwo = embBeads[two];

        // We get the types and subtypes
        string level = levels[typeMappings[beadOne.TypeAndSubType], typeMappings[beadTwo.TypeAndSubType]];
        return interactionLevels[level]*1000; // returns in J/mol
    }
    
    private static float interactionLevel(EmbeddedBead beadOne, EmbeddedBead beadTwo) {

        // We get the types and subtypes
        string level = levels[typeMappings[beadOne.TypeAndSubType], typeMappings[beadTwo.TypeAndSubType]];
        return interactionLevels[level]*1000; // returns in J/mol
    }

    private float effectiveSize(GameObject one, GameObject two) {
        // We get the embedded info
        EmbeddedBead beadOne = embBeads[one];
        EmbeddedBead beadTwo = embBeads[two];

        if (beadOne.Type == "Q" && beadTwo.Type == "Q")
            return (0.62f)*10; // * Mathf.Pow(10, -9));
        else if ((beadOne.TypeAndSubType == ("C", "1") || beadOne.TypeAndSubType == ("C", "2"))
                 && (beadTwo.TypeAndSubType == ("C", "1") || beadTwo.TypeAndSubType == ("C", "2")))
            return (0.62f)*10;// * Mathf.Pow(10, -9));
        else if (beadOne.TypeAndSubType == ("BP", "4") || beadTwo.TypeAndSubType == ("BP", "4"))
            return (0.57f*10); // * Mathf.Pow(10, -9));
        else
            return (0.47f*10);// * Mathf.Pow(10, -9));

    }
    
    private static float effectiveSize(EmbeddedBead beadOne, EmbeddedBead beadTwo) {

        if (beadOne.Type == "Q" && beadTwo.Type == "Q")
            return (0.62f)*10; // * Mathf.Pow(10, -9));
        else if ((beadOne.TypeAndSubType == ("C", "1") || beadOne.TypeAndSubType == ("C", "2"))
                 && (beadTwo.TypeAndSubType == ("C", "1") || beadTwo.TypeAndSubType == ("C", "2")))
            return (0.62f)*10;// * Mathf.Pow(10, -9));
        else if (beadOne.TypeAndSubType == ("BP", "4") || beadTwo.TypeAndSubType == ("BP", "4"))
            return (0.57f*10); // * Mathf.Pow(10, -9));
        else
            return (0.47f*10);// * Mathf.Pow(10, -9));
    }

    public static Vector3 F_nb(GameObject tgtBead, List<GameObject> exclude, List<GameObject> beads)
    {
        var F_nb = Vector3.zero;
        var pos = beads.Select(x => x.transform.position).ToList();
        var idx = beads.IndexOf(tgtBead);
        
        foreach (var (bead, i) in beads.Except(exclude).Select((x, i) => (x, i)))
        {
            var dir = (pos[i] - pos[idx]).normalized;
            var distance = (pos[i] - pos[idx]).magnitude;
            var interactionLevel = NonBondedForce.interactionLevel(tgtBead.GetComponent<EmbeddedBead>(), bead.GetComponent<EmbeddedBead>());
            var effectiveSize = NonBondedForce.effectiveSize(tgtBead.GetComponent<EmbeddedBead>(), bead.GetComponent<EmbeddedBead>());

            F_nb += F_LJ(distance, effectiveSize, interactionLevel) * dir;
        }

        return F_nb;
    }

    // Start is called before the first frame update
    /*
    void Start()
    {
        embBeads[gameObject] = gameObject.GetComponent<EmbeddedBead>();
        
        List<GameObject> neighbours = new List<GameObject>();
        // I have to revise how to scale the value of the radius to a real world bead
        // Each bead, technically, has a radius of at most σ = 1.2nm
        Collider[] others = Physics.OverlapSphere(transform.position, 12f * (float) Math.Pow(10, -9));  

        // Takes from all the nearby colliders, only the ones that are beads
        // and then stores them in a list of GameObjects
        foreach (var other in others) {
            if (other.gameObject.tag == "Bead") {
                neighbours.Add(other.gameObject);
                embBeads[other.gameObject] = other.gameObject.GetComponent<EmbeddedBead>();
            }
        }

        // Initialize the register of potentials and distances to all 0s
        potentials = new List<float>(new float[neighbours.Count]);
        distances = new List<float>(new float[neighbours.Count]);
    }
    */

    // FixedUpdate is called in time with the physics simulation
    // using Update will break this code.
    void FixedUpdate()
    {
        embBeads[gameObject] = gameObject.GetComponent<EmbeddedBead>();
        
        List<GameObject> neighbours = new List<GameObject>();
        // I have to revise how to scale the value of the radius to a real world bead
        // Each bead, technically, has a radius of at most σ = 1.2nm
        Collider[] others = Physics.OverlapSphere(transform.position, 12f /* * (float) Math.Pow(10, -9)*/);  

        // Takes from all the nearby colliders, only the ones that are beads
        // and then stores them in a list of GameObjects
        foreach (var other in others) {
            if (other.gameObject.tag == "Bead" && other.gameObject != gameObject) {
                neighbours.Add(other.gameObject);
                embBeads[other.gameObject] = other.gameObject.GetComponent<EmbeddedBead>();
            }
        }

        // Initialize the register of potentials and distances to all 0s
        potentials = new List<float>(new float[neighbours.Count]);
        distances = new List<float>(new float[neighbours.Count]);
        
        for (int i = 0; i < neighbours.Count; i++) {
            GameObject current = this.gameObject;
            GameObject neighbour = neighbours[i];
        
            float C = 1000  * (1 / CONSTANTS.AVOGADRO_NUM) * 10e20f;
            float epsilon = (float) interactionLevel(current, neighbour)*C;
            float sigma = (float) effectiveSize(current, neighbour);
            float distance = Vector3.Distance(transform.position, neighbour.transform.position);
        
            float U_LJ = 4 * epsilon * ((float) Math.Pow((sigma / distance), 12) - (float) Math.Pow((sigma / distance), 6));

            // TODO: implement the derivative
            // double F_nb = (potentials[i] - U_LJ)/(distance - distances[i]);

            var F_nb = F_LJ(distance, sigma, epsilon);
            Vector3 dir = (neighbour.transform.position - transform.position).normalized;
            var F_vec = dir * (float)F_nb;
            
            if (F_vec.ToString().Contains("NaN"))
            {
                continue;
            }

            distances[i] = distance;
            potentials[i] = U_LJ;
            
            current.GetComponent<Rigidbody>().AddForce(dir * (float)F_nb);
            neighbour.GetComponent<Rigidbody>().AddForce(-dir * (float)F_nb);
        }
    }


}
```

---

Repurposing `EmbeddedBead.cs`

```c#
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public enum MyType {
        START, END, NONE
    }

public class EmbeddedBead : MonoBehaviour
{
    // The type will be assigned to be either P (Polar), N (Semipolar), A (Apolar), and Q (Charged) and BP (only for antifreeze particles)
    public string Type;

    // The type will be assigned to be one of two:
    //       * Hydrogen-bonding: a (acceptor), d (donor), da (both), 0 (none)
    //       * Polarity Level: 1 (lowest), 2, 3, 4, 5 (highest)
    public string SubType;

    public string Size = "R";

    public int Index;

    public MyType Cat = MyType.NONE; 

    public (string, string) TypeAndSubType
    {
        get => (Type, SubType);
    }
}
```